In [1]:
from catboost import CatBoostRegressor
from collections import defaultdict
import copy
import datetime
from imp import reload
from mymodule import make_combo_fp, predict_synergy
import numpy as np
import pandas as pd
import pickle
from progiter import ProgIter
from scipy.stats.stats import pearsonr
from sklearn import linear_model
from sklearn.svm import SVR, LinearSVR
from sklearn import neighbors
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.tree import DecisionTreeRegressor
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_validate, cross_val_score
from xgboost import XGBRegressor
import time
import warnings


#from e3fp.pipeline import fprints_from_smiles # for 3D ECFP-like fingerprints

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

def scoring_func_p(y_true, y_pred):
    return pearsonr(y_true, y_pred)[0]
def scoring_func_m(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

pearson = make_scorer(score_func=scoring_func_p, greater_is_better=True)
rmse = make_scorer(score_func=scoring_func_m, greater_is_better=False)


In [122]:
sklearn_all_dgi = make_combo_fp.DataInput(low=0, 
                                      high=300, 
                                      bitaverage=False,
                                      fps_name='/tf/notebooks/code_for_pub/fp_files/fps_infomax_new.pickle')
df_dgi = sklearn_all_dgi.calc_synergy()
inp = df_dgi.copy() # this is for feeding the training loop


dropped 8722844 single drugs
elapsed time: 42.32 seconds
dropped 297740 rows after mapping to what we have in fps
shrank dataset from 8251264 to 362635 rows in 84.48 seconds
creating 5 datasets packed into dict with metrics as key names 5/5... 


'fps_infomax_new.pickle'

In [2]:
sklearn_all = make_combo_fp.DataInput(low=0, 
                                      high=1024, 
                                      bitaverage=False,
                                      fps_name='/tf/notebooks/code_for_pub/fp_files/fps_morgan_1024bit.pickle')
df = sklearn_all.calc_synergy()
inp = df.copy() # this is for feeding the training loop


dropped 8722844 single drugs
elapsed time: 137.77 seconds
dropped 297740 rows after mapping to what we have in fps
shrank dataset from 8251264 to 362635 rows in 55.81 seconds
creating 5 datasets packed into dict with metrics as key names 5/5... 


In [ ]:
# GPU models are not friends with sklearn pipeline very much
from sklearn.model_selection import KFold
from catboost import Pool, CatBoostRegressor
from scipy.stats import pearsonr
from progiter import ProgIter
import warnings 

X=0.01
inp.pop('name',None) # removing name of fp used to create averaged across experimental replicates dataset
big_holder = []

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for i in ProgIter(range(10), desc='training sklearn models + xgboost + catboost\n'):

        folds = KFold(n_splits=5, random_state=i+1, shuffle=True)
        holder = dict() # store fold results
        for key, value in inp.items():
            
            # this column was used to group replicate experiments across studies in order to calculate average synergy
            if 'drug_row_col' in value.columns:
                value.drop(columns=['drug_row_col'], inplace=True)
                
            # lists to hold current synergy types results results
            ## gpr = [] for gaussian processes above ~30_000-40_000 rows it is too long
            cbt,rfr,xgb,gbr,lr,ridge,lasso,elastic,lassolars,b_ridge,svr = [],[],[],[],[],[],[],[],[],[],[]
            for train_ind, test_ind in folds.split(value):
                print(f'{key} with fold: {len(cbt)}')
                
                # catboost part. he is not friends with cross val
                y_train = value.loc[train_ind][key]
                x_train = value[[x for x in value.columns if x not in key]].loc[train_ind]
                y_test = value.loc[test_ind][key]
                x_test = value[[x for x in value.columns if x not in key]].loc[test_ind]
                
                # downsampling for cbt
                x_train = x_train.sample(frac=X, random_state=23)
                y_train = y_train.loc[x_train.index]
                x_test = x_test.sample(frac=X, random_state=23)
                y_test = y_test.loc[x_test.index]

                model = CatBoostRegressor(task_type='GPU')
                train_dataset = Pool(data=x_train, label=y_train, cat_features=['cell_line_name'])
                eval_dataset = Pool(data=x_test, label=y_test, cat_features=['cell_line_name'])
                model.fit(train_dataset, verbose=0)
                pred = model.predict(eval_dataset)
                score_cbt = pearsonr(y_test, pred)[0]
                cbt.append(score_cbt)
                del model

                ######
                # sklearn loop
                model_xgb = XGBRegressor(tree_method='gpu_hist')
                model_rfr = RandomForestRegressor(warm_start=True,verbose=0)
                model_gbr = HistGradientBoostingRegressor(warm_start=True,verbose=0)
                model_lr = linear_model.LinearRegression()
                model_ridge = linear_model.Ridge()
                model_lasso = linear_model.Lasso()
                model_elastic = linear_model.ElasticNet()
                model_lassolars = linear_model.LassoLars()
                model_b_ridge = linear_model.BayesianRidge()
                #kernel = DotProduct() + WhiteKernel()
                #model_gpr = GaussianProcessRegressor(kernel=kernel)
                model_linSVR = LinearSVR()

                full = pd.get_dummies(value, columns=['cell_line_name'])
                x_train = full[[x for x in full.columns if key not in x]].loc[train_ind]
                y_train = full.loc[train_ind][key]
                x_test = full[[x for x in full.columns if key not in x]].loc[test_ind]
                y_test = full.loc[test_ind][key]

                # downsmapling for sklearn loop
                x_train = x_train.sample(frac=X, random_state=i)
                y_train = y_train.loc[x_train.index]
                x_test = x_test.sample(frac=X, random_state=i)
                y_test = y_test.loc[x_test.index]

                # individual model's loop
                for model,l,name in zip([model_xgb,model_rfr,model_gbr,model_lr,model_ridge,model_lasso,
                                         model_elastic,model_lassolars,model_b_ridge,model_linSVR], 
                                         [xgb,rfr,gbr,lr,ridge,lasso,elastic,lassolars,b_ridge,svr], 
                                         ['xgb', 'rfr', 'gbr', 'model_lr','model_ridge','model_lasso',
                                          'model_elastic','model_lassolars','model_b_ridge','model_linSVR']):

                    model.fit(x_train, y_train)
                    pred = model.predict(x_test)
                    score = pearsonr(y_test, pred)[0]
                    l.append(score)
                    del model
            
            # aggregate kfold results for a given range iteration
            holder[key]={ 
                k:np.nanmean(v) for k,v in zip(
                    ['cbt','xgb','rfr','gbr','model_lr','model_ridge','model_lasso',\
                     'model_elastic','model_lassolars','model_b_ridge','model_linSVR'],
                    [cbt,xgb,rfr,gbr,lr,ridge,lasso,elastic,lassolars,b_ridge,svr])
                      }
        # aggregate in the big thing
        big_holder.append(pd.DataFrame(holder))
        # saving infomax
        with open(f'/tf/notebooks/code_for_pub/_logs_as_python_files/_0_compare_11_models_against_Morgan_fold0to{i}_with_0.01DATA_5foldMODELS.pickle', 'wb') as f:
            pickle.dump(big_holder, f)



    


training sklearn models + xgboost + catboost
  0/10... rate=0 Hz, eta=?, total=0:00:00, wall=01:01 UTCcss_ri with fold: 0
css_ri with fold: 1


In [4]:
# ranking on morgan

with open('/tf/notebooks/code_for_pub/_logs_as_python_files/_0_compare_11_models_against_Morgan_fold0to1_with_0.01DATA_5foldMODELS.pickle', 'rb') as f:
    morgan = pickle.load(f)

results_morgan = pd.DataFrame()

for df in morgan:
    results_morgan = pd.concat([results_morgan, df],axis=1)

for cols in results_morgan.columns.drop_duplicates():
    results_morgan[cols] = results_morgan[cols].mean(axis=1)
results_morgan = results_morgan.loc[:,~results_morgan.columns.duplicated()]


In [13]:
# saving infomax
#with open('/tf/notebooks/code_for_pub/_logs_as_python_files/_0_compare_11_models_against_Infomax_3folds_with_0.01DATA_5foldMODELS.pickle', 'wb') as f:
#    pickle.dump(big_holder, f)

In [2]:
# ranking on infomax

with open('/tf/notebooks/code_for_pub/_logs_as_python_files/_0_compare_11_models_against_Infomax_3folds_with_0.01DATA_5foldMODELS.pickle', 'rb') as f:
    infomax = pickle.load(f)

results_infomax = pd.DataFrame()

for df in infomax:
    results_infomax = pd.concat([results_infomax, df],axis=1)

for cols in results_infomax.columns.drop_duplicates():
    results_infomax[cols] = results_infomax[cols].mean(axis=1)
results_infomax = results_infomax.loc[:,~results_infomax.columns.duplicated()]

#print(results_infomax.sum(axis=1).sort_values(ascending=False))

In [6]:
results = pd.DataFrame()
for df in [results_infomax, results_morgan]: # aggregate infomax dfs
    results = pd.concat([results, df],axis=1) 
for cols in results.columns.drop_duplicates(): # get column-wise mean
    results[cols] = results[cols].mean(axis=1)
results = results.loc[:,~results.columns.duplicated()]
backup=results.copy()

for x in results.columns:
    results[x] = results[x].rank(ascending=False) # rank based on each column
results['rank'] = results.mean(axis=1) # get aggregate rank
results.iloc[:,:-1] = backup.iloc[:,:] # put back real values
results['rank'] = results['rank'].rank(ascending=True) # leave average column to rank by
results = results.sort_values(by='rank')
results.drop(labels='model_lassolars', axis=0, inplace=True) 
results['rank'] = results['rank'].astype(int)

In [11]:
print(np.round(results, decimals=3).to_latex(bold_rows=True, longtable=False, index=False ))

\begin{tabular}{rrrrrr}
\toprule
 css\_ri &  synergy\_zip &  synergy\_bliss &  synergy\_loewe &  synergy\_hsa &  rank \\
\midrule
  0.641 &        0.384 &          0.331 &          0.384 &        0.303 &     1 \\
  0.609 &        0.413 &          0.355 &          0.374 &        0.311 &     2 \\
  0.610 &        0.419 &          0.339 &          0.333 &        0.244 &     3 \\
  0.624 &        0.373 &          0.316 &          0.345 &        0.265 &     4 \\
  0.616 &        0.367 &          0.315 &          0.299 &        0.283 &     5 \\
  0.588 &        0.333 &          0.270 &          0.253 &        0.219 &     6 \\
  0.599 &        0.311 &          0.251 &          0.287 &        0.219 &     7 \\
  0.332 &          NaN &            NaN &          0.185 &          NaN &     8 \\
  0.092 &        0.030 &          0.023 &          0.042 &        0.022 &     9 \\
  0.264 &          NaN &            NaN &            NaN &          NaN &    10 \\
\bottomrule
\end{tabular}



In [8]:
results

,css_ri,synergy_zip,synergy_bliss,synergy_loewe,synergy_hsa,rank
gbr,0.641193,0.384281,0.331276,0.384461,0.303057,1
rfr,0.608859,0.413155,0.354832,0.374183,0.310755,2
cbt,0.609812,0.419181,0.339147,0.333218,0.243566,3
xgb,0.623941,0.373301,0.316269,0.344636,0.265482,4
model_b_ridge,0.616473,0.366547,0.314767,0.299176,0.283198,5
model_linSVR,0.587912,0.332821,0.270043,0.252639,0.218849,6
model_ridge,0.599370,0.310586,0.250538,0.286958,0.218554,7
model_elastic,0.331760,NaN,NaN,0.184714,NaN,8
model_lr,0.091675,0.030127,0.022943,0.041568,0.022207,9
model_lasso,0.264287,NaN,NaN,NaN,NaN,10


In [37]:
results

,css_ri,synergy_zip,synergy_bliss,synergy_loewe,synergy_hsa,average
cbt,4.0,1.0,2.0,4.0,5.0,3.2
xgb,2.0,4.0,4.0,3.0,4.0,3.4
rfr,5.0,2.0,1.0,2.0,1.0,2.2
gbr,1.0,3.0,3.0,1.0,2.0,2.0
model_lr,10.0,8.0,8.0,9.0,8.0,8.6
model_ridge,6.0,7.0,7.0,6.0,7.0,6.6
model_lasso,9.0,NaN,NaN,NaN,NaN,9.0
model_elastic,8.0,NaN,NaN,8.0,NaN,8.0
model_lassolars,NaN,NaN,NaN,NaN,NaN,NaN
model_b_ridge,3.0,5.0,5.0,5.0,3.0,4.2


In [47]:
pd.DataFrame(holder).sum(axis=1, skipna=False).sort_values(ascending=False)

cbt          2.034857
gbr          1.953725
xgb          1.880798
rfr          1.732182
ridge        1.664641
bridge       1.640975
svr          1.496834
gpr          1.455512
lr           0.959521
lasso             NaN
elastic           NaN
lassolars         NaN
dtype: float64

## old work. sklearn cross_validate is not friends with catboost loader

In [11]:
def run_train_all_sklearn(file, fp_name, cv=5, verbose=0, seed=1):
    
    np.random.seed(seed)
    c = defaultdict(list)
    
    for k in ProgIter(['synergy_zip', 'synergy_bliss', 'synergy_loewe', 'synergy_hsa','css_ri','name'], verbose=verbose, total=5):
        v = file[k]
        
        if k != 'name':
            temp = dict() # for results storage. Assuming that "name" comes last
            
            if 'drug_row_col' in v.columns:
                v.drop(columns=['drug_row_col'], inplace=True)
            
            cat_cols = ['cell_line_name']
            categories = [v[column].unique() for column in v[cat_cols]] # manually find all available categories for one-hot
            
            # pipelines
            encode = Pipeline(steps=[
                ('one-hot-encode', OneHotEncoder(categories=categories))
            ])
            processor = ColumnTransformer(
                transformers=[
                    ('cat_encoding', encode, cat_cols),
                    ('dropping', 'drop', [k]) 
                ],
                remainder='passthrough')
            
            catbst = ColumnTransformer(
                transformers=[
                    ('dropping', 'drop', [k]) 
                ],
                remainder='passthrough')
            
            # regressions
            lr = make_pipeline(processor, linear_model.LinearRegression())
            ridge = make_pipeline(processor, linear_model.Ridge())
            lasso = make_pipeline(processor, linear_model.Lasso())
            elastic = make_pipeline(processor, linear_model.ElasticNet())
            lassolars = make_pipeline(processor, linear_model.LassoLars())
            b_ridge = make_pipeline(processor, linear_model.BayesianRidge())
            kernel = DotProduct() + WhiteKernel()
            gpr = make_pipeline(processor, GaussianProcessRegressor(kernel=kernel))
            linSVR = make_pipeline(processor, LinearSVR())
            hist_gbr = make_pipeline(processor, HistGradientBoostingRegressor(warm_start=True, 
                                                                              max_depth=6))
            rfr = make_pipeline(processor, RandomForestRegressor(warm_start=True, 
                                                                 max_depth=6, 
                                                                 n_jobs=3))
            iso = make_pipeline(processor, IsotonicRegression(increasing='auto'))
            xgb = make_pipeline(processor, XGBRegressor(tree_method='gpu_hist', 
                                                        max_depth=6))
            cbt = make_pipeline(catbst, CatBoostRegressor(task_type='GPU', 
                                                          depth=6, 
                                                          cat_features=np.array([0]), 
                                                          verbose=False))
            
            mls = [
                   lr, ridge, lasso, elastic, lassolars, 
                   b_ridge, 
                   gpr, 
                   linSVR, 
                   iso]
            mls_names = [
                         "lr", "ridge","lasso","elastic","lassolars", 
                         "b_ridge", 
                         "gpr", 
                         "linSVR", 
                         "iso"]
            #cbt{'test_pearsonr': 0.4059569639624347, 'test_rmse': 3.5055945236237767}
            #rfr{'test_pearsonr': 0.321618212063255, 'test_rmse': 3.649483632747507}
            # gpr - nan
            # hist_gbr {'test_pearsonr': 0.43053228829409884, 'test_rmse': 3.469798635399871}
            # xgb 0.43646047338496075, 'test_rmse': 3.449668513175836}
            # results 
            start = time.time()
            for MODEL, name in zip(mls, mls_names):
                print(f'\n{name}')
                if ('cbt' == name) | ('hist_gbr' == name) |('hist_gbr' == name) :
                    n_jobs = 1
                else:
                    n_jobs = 1
                cv_dict = cross_validate(
                    MODEL,
                    v, 
                    v[k],
                    cv=cv, 
                    scoring={"pearsonr":pearson,
                             "rmse":rmse}, 
                    return_train_score=False, 
                    verbose=verbose,
                    n_jobs=n_jobs, 
                )
                temp[name] = {
                    'test_pearsonr' : np.nanmean(cv_dict['test_pearsonr']), 
                    'test_rmse' : abs(np.nanmean(cv_dict['test_rmse']))
                }
                print(temp[name])
            print(f'{k} took {int(time.time()-start)/60} mins')
            
            c[k] = temp
        else:
            nm=f'/tf/notebooks/code_for_pub/_logs_as_python_files/{fp_name}_13models_5foldCV_{time.ctime()}.pickle'
            with open(nm, 'wb') as file:
                pickle.dump(c, file)
            print(f'saving complete to {nm}')
    return c
   

In [3]:
sklearn_all = make_combo_fp.DataInput(low=0, 
                                      high=1024, 
                                      bitaverage=False,
                                      fps_name='/tf/notebooks/code_for_pub/fp_files/fps_morgan_1024bit.pickle')
df = sklearn_all.calc_synergy()

dropped 8722844 single drugs
elapsed time: 84.13 seconds
dropped 297740 rows after mapping to what we have in fps
shrank dataset from 8251264 to 362635 rows in 59.12 seconds
creating 5 datasets packed into dict with metrics as key names 5/5... 


In [12]:
results_morgan = run_train_all_sklearn(df, 'Morgan1024', verbose=1)

 0/5... rate=0 Hz, eta=?, total=0:00:00, wall=13:59 UTC
lr


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  9.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


{'test_pearsonr': 0.06793247093176949, 'test_rmse': 3097330246952.0283}

ridge


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  3.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


{'test_pearsonr': 0.31484287823616997, 'test_rmse': 3.8757620066923844}

lasso


...

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  4.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


{'test_pearsonr': 0.5785603568199782, 'test_rmse': 21.678422093849676}

iso


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/isotonic.py", line 313, in fit
    X, y = self._build_y(X, y, sample_weight)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/isotonic.py", line 237, in _build_y
    self._check_fit_data(X, y, sample_weight)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/isotonic.py", line 216, in _check_fit_data
    raise ValueError("X should be a 1d array")
ValueError: X should be a

{'test_pearsonr': nan, 'test_rmse': nan}
css_ri took 50.5 mins
 5/5... rate=0.000296719 Hz, eta=0:00:00, total=4:40:50, wall=18:40 UTCsaving complete to /tf/notebooks/code_for_pub/_logs_as_python_files/Morgan1024_13models_5foldCV_Thu Dec 31 18:40:32 2020.pickle
 6/5... rate=0.000356063 Hz, eta=-1 day, 23:13:12, total=4:40:50, wall=18:40 UTC


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/isotonic.py", line 313, in fit
    X, y = self._build_y(X, y, sample_weight)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/isotonic.py", line 237, in _build_y
    self._check_fit_data(X, y, sample_weight)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/isotonic.py", line 216, in _check_fit_data
    raise ValueError("X should be a 1d array")
ValueError: X should be a

In [15]:
keys, values = zip(*results_morgan.items())
keys1, values1 = zip(*values[0].items())
holder_r = pd.DataFrame(index=keys1, columns=keys)
holder_rmse = pd.DataFrame(index=keys1, columns=keys)

In [17]:
for metric,v in results_morgan.items():
    for model, values in v.items():
        holder_r.loc[model, metric] = values['test_pearsonr']
        holder_rmse.loc[model, metric] = values['test_rmse']

holder_r['rank']= holder_r.rank(
    axis='rows',
    na_option='bottom', 
    method='dense', 
    ascending=False
).apply(
    np.mean,axis='columns'
).rank(
    method='first'
)
holder_rmse['rank']= holder_rmse.rank(axis='rows', 
                              na_option='bottom', 
                              method='dense', 
                              ascending=True).apply(np.mean, 
                                                     axis='columns').rank(method='first')
holder_r.sort_values(by='rank', inplace=True)
holder_rmse.sort_values(by='rank', inplace=True)

In [13]:
results_morgan

defaultdict(list,
            {'synergy_zip': {'lr': {'test_pearsonr': 0.06793247093176949,
               'test_rmse': 3097330246952.0283},
              'ridge': {'test_pearsonr': 0.31484287823616997,
               'test_rmse': 3.8757620066923844},
              'lasso': {'test_pearsonr': nan, 'test_rmse': 3.9522534677179153},
              'elastic': {'test_pearsonr': nan,
               'test_rmse': 3.9522534677179153},
              'lassolars': {'test_pearsonr': nan,
               'test_rmse': 3.9522534677179153},
              'b_ridge': {'test_pearsonr': 0.32345110696401624,
               'test_rmse': 3.8195297642691592},
              'gpr': {'test_pearsonr': nan, 'test_rmse': nan},
              'linSVR': {'test_pearsonr': 0.32603547331593496,
               'test_rmse': 3.799658557918773},
              'iso': {'test_pearsonr': nan, 'test_rmse': nan}},
             'synergy_bliss': {'lr': {'test_pearsonr': 0.0643324480450066,
               'test_rmse': 3065487187148.579}

In [86]:
import os
os.chdir('/tf/notebooks/code_for_pub')
print(os.getcwd())


/tf/notebooks/code_for_pub


In [87]:

holder_rmse.to_csv('rmse_all_models_test_sklearn.csv')
holder_r.to_csv('r_all_models_test_sklearn.csv')

In [88]:
with open('/tf/notebooks/code_for_pub/_logs_as_python_files/infomax_300bit_noReplicates_14models_5foldCV_Wed Dec 16 07:36:02 2020.pickle', 'rb') as f:
    inf = pickle.load(f)

In [91]:
keys, values = zip(*inf.items())
keys1, values1 = zip(*values[0].items())
holder_r = pd.DataFrame(index=keys1, columns=keys)
holder_rmse = pd.DataFrame(index=keys1, columns=keys)

In [92]:
for metric,v in output.items():
    for model, values in v.items():
        holder_r.loc[model, metric] = values['test_pearsonr']
        holder_rmse.loc[model, metric] = values['test_rmse']

holder_r['rank']= holder_r.rank(
    axis='rows',
    na_option='bottom', 
    method='dense', 
    ascending=False
).apply(
    np.mean, axis='columns'
).rank(
    method='first'
)
holder_rmse['rank']= holder_rmse.rank(axis='rows', 
                              na_option='bottom', 
                              method='dense', 
                              ascending=True).apply(np.mean, 
                                                     axis='columns').rank(method='first')
holder_r.sort_values(by='rank', inplace=True)
holder_rmse.sort_values(by='rank', inplace=True)

In [93]:

holder_rmse.to_csv('rmse_all_models_test_sklearn_infomax.csv')
holder_r.to_csv('r_all_models_test_sklearn_infomax.csv')